<a href="https://colab.research.google.com/github/ShainaBagri/ClassifyDocumentTopics/blob/main/proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!wget 'http://users.csc.calpoly.edu/~foaad/proj1F21_files.zip'
!unzip 'proj1F21_files.zip'

--2021-10-20 19:55:41--  http://users.csc.calpoly.edu/~foaad/proj1F21_files.zip
Resolving users.csc.calpoly.edu (users.csc.calpoly.edu)... 129.65.128.20
Connecting to users.csc.calpoly.edu (users.csc.calpoly.edu)|129.65.128.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129006 (126K) [application/zip]
Saving to: ‘proj1F21_files.zip.1’

proj1F21_files.zip. 100%[===================>] 125.98K   545KB/s    in 0.2s    

2021-10-20 19:55:42 (545 KB/s) - ‘proj1F21_files.zip.1’ saved [129006/129006]

Archive:  proj1F21_files.zip
replace proj1F21_files/proj1F21_1412_A.html? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: proj1F21_files/proj1F21_1412_A.html  
  inflating: proj1F21_files/proj1F21_1412_B.html  
  inflating: proj1F21_files/proj1F21_1422_A.html  
  inflating: proj1F21_files/proj1F21_1422_B.html  
  inflating: proj1F21_files/proj1F21_1483_A.html  
  inflating: proj1F21_files/proj1F21_1483_B.html  
  inflating: proj1F21_files/proj1F21_1794_A.html  
  

In [79]:
!pip install html2text

In [151]:
import nltk
nltk.download('punkt')
# from nltk import PunktSentenceTokenizer
import pandas as pd
import numpy as np
import os
import random
import re
import html2text
from sklearn.feature_extraction.text import TfidfVectorizer
random.seed(123)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [152]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def fixRaw(text):
  newText = ""
  for p in re.split('\n\n+', text):
    newText = newText + p.replace('\n', ' ').strip() + '\n'
  return newText

def tokenizeText(text):
  goodParas = []
  for para in re.split('\n\n+', text):
    sents = sent_tokenizer.tokenize(para.replace('\n', ' ').strip())
    goodSents = []
    for sent in sents:
      # TODO: lemmatize / convert contractions here?
      words = [word.lower() for word in nltk.word_tokenize(sent)]
      goodSents.append(words)
    
    goodParas.append(goodSents)
      
  return goodParas

def extractFeaturesDoc(tokText):
 
  features = {}

  # dummy data
  features['len'] = len(tokText)

  # features['tfidf'] = tfidf

  return features

In [153]:
dir = './proj1F21_files/'
# docs = pd.DataFrame(columns=['author', 'subject', 'rawText'])
docs = []
for document in os.listdir(dir):
  dict = {}
  docName = document.split("_")
  dict['author'] = docName[1]
  dict['subject'] = docName[2][0]
  with open(dir + document) as f:
    dict['htmlText'] = f.read()
    dict['rawText'] = html2text.html2text(dict['htmlText'])
    dict['fixedRawText'] = fixRaw(dict['rawText'])
    # tokenize 
    dict['tokenizedText'] = tokenizeText(dict['rawText']) 

  docs.append(dict)

# split into training and test sets
random.shuffle(docs)
split = int(len(docs) *0.8)
test_set = pd.DataFrame(docs[split:])
training_set = pd.DataFrame(docs[:split])

# print(training_set['htmlText'][0])
# print(training_set['rawText'][0])
print(training_set['tokenizedText'][0])

[[['while', 'i', 'haven', '’', 't', 'had', 'a', 'ton', 'of', 'experiences', 'to', 'shape', 'myself', 'into', 'a', 'leader', 'yet', ',', 'i', 'know', 'that', 'one', 'leader', 'i', 'look', 'up', 'to', 'a', 'lot', 'is', 'steve', 'jobs', '.'], ['while', 'steve', 'jobs', 'wasn', '’', 't', 'the', 'best', 'person', 'in', 'the', 'world', 'and', 'did/said', 'a', 'lot', 'of', 'mean', 'and', 'hurtful', 'things', 'to', 'other', 'people', ',', 'i', 'think', 'there', 'were', 'certain', 'aspects', 'of', 'his', 'leadership', 'style', 'that', 'i', 'really', 'admire', 'and', 'would', 'like', 'to', 'emulate', '.'], ['there', 'are', 'many', 'leadership', 'traits', 'that', 'steve', 'jobs', 'embodied', ',', 'and', 'i', 'would', 'like', 'to', 'embody', 'as', 'well', '.'], ['i', 'think', 'the', 'most', 'significant', 'three', 'traits', 'were', 'decisiveness', ',', 'tact', ',', 'and', 'endurance', '.'], ['as', 'apple', '’', 's', 'ceo', ',', 'steve', 'jobs', 'constantly', 'made', 'decisions', '.'], ['he', 'made

In [154]:
print(training_set['rawText'][0])
print(training_set['fixedRawText'][0])

    While I haven’t had a ton of experiences to shape myself into a leader yet, I know that one leader I look up to a lot is Steve Jobs. While Steve Jobs wasn’t the best person in the world and did/said a lot of mean and hurtful things to other people, I think there were certain aspects of his leadership style that I really admire and would like to emulate. There are many leadership traits that Steve Jobs embodied, and I would like to embody as well. I think the most significant three traits were Decisiveness, Tact, and Endurance.  
As Apple’s CEO, Steve Jobs constantly made decisions. He made decisions to
determine the aesthetics of Apple products, what kinds of markets they should
target, very small details of product design, as well as many other decisions.
The majority of the decisions Steve Jobs made were made very quickly, and most
of the time, were also the best decisions to be made. I am reading his
biography right now, and there are multiple accounts of people coming to work
f

In [118]:
## TFIDF needs to be outside features bec. it needs to be fit on all documents, not each one separately
# tfidf = TfidfVectorizer(sublinear_tf=True, max_df=0.75)
# tfidf.fit(training_set['fixedRawText'])
# training_set['tfidf'] = tfidf.transform(training_set['fixedRawText'])

In [155]:
training_set['features'] = training_set.apply(lambda x: extractFeaturesDoc(x['tokenizedText']), axis=1)
training_set.head()

,author,subject,htmlText,rawText,fixedRawText,tokenizedText,features
0,8842,A,"\n\n\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML...",While I haven’t had a ton of experiences t...,While I haven’t had a ton of experiences to sh...,"[[[while, i, haven, ’, t, had, a, ton, of, exp...",{'len': 5}
1,3647,B,"\n\n\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML...","Although somewhat rare in California, Subaru O...","Although somewhat rare in California, Subaru O...","[[[although, somewhat, rare, in, california, ,...",{'len': 6}
2,2605,A,"\n\n\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML...",My favorite person is my high school headmaste...,My favorite person is my high school headmaste...,"[[[my, favorite, person, is, my, high, school,...",{'len': 7}
3,2592,A,"\n\n\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML...",One of my favorite people is Satoshi Nakamoto....,One of my favorite people is Satoshi Nakamoto....,"[[[one, of, my, favorite, people, is, satoshi,...",{'len': 6}
4,6388,B,"\n\n\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML...",Old Betsy was a reliable Green Honda Accord th...,Old Betsy was a reliable Green Honda Accord th...,"[[[old, betsy, was, a, reliable, green, honda,...",{'len': 6}


In [ ]:
# from sklearn.model_selection import cross_val_score

# cv_scores = cross_val_score(naiveBayes, X_train, y_train, cv=10, scoring="accuracy")
# print(cv_scores.mean())

In [176]:
num_folds = 10
subset = int(len(training_set)/num_folds)
nbAccuracies = []
for i in range(num_folds):
    test_fold = training_set.loc[i*subset:(i+1)*subset]
    train_fold = pd.concat([training_set.loc[:i*subset], training_set.loc[(i+1)*subset:]])
    featureset_train = list(zip(train_fold['features'],train_fold['subject']))
    featureset_test = list(zip(test_fold['features'],test_fold['subject']))
    naiveBayes = nltk.classify.NaiveBayesClassifier.train(featureset_train)
    # decisionTree = nltk.classify.DecisionTreeClassifier()
    nbAccuracies.append(nltk.classify.accuracy(naiveBayes, featureset_test))
print(sum(nbAccuracies)/len(nbAccuracies))

0.5833333333333334
